In [172]:
from initial_pop import *
from code_classes import *

In [173]:
class Player:
    def __init__(self, name, position, skill, cost):
        self.name = name
        self.position = position
        self.skill = skill
        self.cost = cost

    def __str__(self):
        return f"{self.name} ({self.position}) - Skill: {self.skill}, Cost: {self.cost}M"


In [184]:
class Team:
    def __init__(self, players):
        self.players = players  # players is a list of Player objects
        self.validate_team()

    def validate_team(self):
        positions = {"GK": 0, "DEF": 0, "MID": 0, "FWD": 0}
        for player in self.players:
            if player.position not in positions:
                raise ValueError(f"Invalid player position: {player.position}")
            positions[player.position] += 1

        # Check the required structure
        if positions["GK"] != 1 or positions["DEF"] != 2 or positions["MID"] != 2 or positions["FWD"] != 2:
            raise ValueError("Each team must have 1 GK, 2 DEF, 2 MID, and 2 FWD.")

        # Check if the team exceeds salary cap
        total_salary = sum(player.cost for player in self.players)
        if total_salary > 750:
            raise ValueError(f"Team salary exceeds the cap: {total_salary}M")

    def get_average_skill(self):
        return sum(player.skill for player in self.players) / len(self.players)

    def __str__(self):
        return "\n".join([str(player) for player in self.players])

In [ ]:
class League:
    def __init__(self, teams):
        self.teams = teams
        self.validate_league()
                
    def validate_league(self):
        if len(self.teams) != 5:
            raise ValueError("The league must have exactly 5 teams.")

        player_names = set()
        for team in self.teams:
            # Explicitly call team.validate_team()
            team.validate_team()

            for player in team.players:
                if player.name in player_names:
                    raise ValueError(f"Player {player.name} is already in another team.")
                player_names.add(player.name)


    def is_valid(self):
        try:
            self.validate_league()
            return True
        except ValueError:
            return False

    def get_standard_deviation_of_average_skills(self):
        avg_skills = [team.get_average_skill() for team in self.teams]
        return np.std(avg_skills)

    def __str__(self):
        return "\n\n".join([str(team) for team in self.teams])

In [ ]:
def calculate_fitness(league):
    if league is None or not league.is_valid():
        return 9999
    return league.get_standard_deviation_of_average_skills()

In [177]:
players = load_players_from_csv("data/players(in).csv")
players

[Alex Carter (GK) Skill: 85 Salary: €90.0M,
 Jordan Smith (GK) Skill: 88 Salary: €100.0M,
 Ryan Mitchell (GK) Skill: 83 Salary: €85.0M,
 Chris Thompson (GK) Skill: 80 Salary: €80.0M,
 Blake Henderson (GK) Skill: 87 Salary: €95.0M,
 Daniel Foster (DEF) Skill: 90 Salary: €110.0M,
 Lucas Bennett (DEF) Skill: 85 Salary: €90.0M,
 Owen Parker (DEF) Skill: 88 Salary: €100.0M,
 Ethan Howard (DEF) Skill: 80 Salary: €70.0M,
 Mason Reed (DEF) Skill: 82 Salary: €75.0M,
 Logan Brooks (DEF) Skill: 86 Salary: €95.0M,
 Caleb Fisher (DEF) Skill: 84 Salary: €85.0M,
 Nathan Wright (MID) Skill: 92 Salary: €120.0M,
 Connor Hayes (MID) Skill: 89 Salary: €105.0M,
 Dylan Morgan (MID) Skill: 91 Salary: €115.0M,
 Hunter Cooper (MID) Skill: 83 Salary: €85.0M,
 Austin Torres (MID) Skill: 82 Salary: €80.0M,
 Gavin Richardson (MID) Skill: 87 Salary: €95.0M,
 Spencer Ward (MID) Skill: 84 Salary: €85.0M,
 Sebastian Perry (FWD) Skill: 95 Salary: €150.0M,
 Xavier Bryant (FWD) Skill: 90 Salary: €120.0M,
 Elijah Sanders 

In [187]:
import random

def create_valid_team_from_pool(player_pool):
    max_attempts = 100
    for _ in range(max_attempts):
        gks = [p for p in player_pool if p.position == "GK"]
        defs = [p for p in player_pool if p.position == "DEF"]
        mids = [p for p in player_pool if p.position == "MID"]
        fwds = [p for p in player_pool if p.position == "FWD"]

        if len(gks) < 1 or len(defs) < 2 or len(mids) < 2 or len(fwds) < 2:
            raise ValueError("Not enough players in the pool to form a valid team.")

        selected_players = random.sample(gks, 1) + \
                           random.sample(defs, 2) + \
                           random.sample(mids, 2) + \
                           random.sample(fwds, 2)

        try:
            team = Team(selected_players)
            return team
        except ValueError:
            continue

    raise ValueError("Failed to create a valid team after many attempts.")

def create_valid_league(all_players, num_teams=5):
    max_attempts = 100
    for _ in range(max_attempts):
        random.shuffle(all_players)
        available_players = all_players.copy()
        used_names = set()
        teams = []

        try:
            for _ in range(num_teams):
                pool = [p for p in available_players if p.name not in used_names]
                team = create_valid_team_from_pool(pool)
                teams.append(team)
                used_names.update(p.name for p in team.players)
            return League(teams)
        except ValueError:
            continue

    raise ValueError("Failed to create a valid league after many attempts.")

def generate_population(players, num_leagues=5):
    population = []
    for _ in range(num_leagues):
        league = create_valid_league(players)
        population.append(league)
    return population


In [188]:
import numpy as np

In [189]:
population = generate_population(players, num_leagues=10)

for i, league in enumerate(population):
    print(f"\n--- League {i+1} ---")
    print(league)
    print(f"Standard Deviation of Avg Skills: {league.get_standard_deviation_of_average_skills():.2f}")


--- League 1 ---
Alex Carter (GK) Skill: 85 Salary: €90.0M
Jaxon Griffin (DEF) Skill: 79 Salary: €65.0M
Ethan Howard (DEF) Skill: 80 Salary: €70.0M
Dominic Bell (MID) Skill: 86 Salary: €95.0M
Nathan Wright (MID) Skill: 92 Salary: €120.0M
Sebastian Perry (FWD) Skill: 95 Salary: €150.0M
Xavier Bryant (FWD) Skill: 90 Salary: €120.0M

Jordan Smith (GK) Skill: 88 Salary: €100.0M
Daniel Foster (DEF) Skill: 90 Salary: €110.0M
Brayden Hughes (DEF) Skill: 87 Salary: €100.0M
Spencer Ward (MID) Skill: 84 Salary: €85.0M
Dylan Morgan (MID) Skill: 91 Salary: €115.0M
Elijah Sanders (FWD) Skill: 93 Salary: €140.0M
Chase Murphy (FWD) Skill: 86 Salary: €95.0M

Chris Thompson (GK) Skill: 80 Salary: €80.0M
Maxwell Flores (DEF) Skill: 81 Salary: €72.0M
Lucas Bennett (DEF) Skill: 85 Salary: €90.0M
Bentley Rivera (MID) Skill: 88 Salary: €100.0M
Connor Hayes (MID) Skill: 89 Salary: €105.0M
Adrian Collins (FWD) Skill: 85 Salary: €90.0M
Colton Gray (FWD) Skill: 91 Salary: €125.0M

Ryan Mitchell (GK) Skill: 83 

## lol this is a mutator :()

In [ ]:
def mutate_within_league_by_position(league):
    positions = ['GK', 'DEF', 'MID', 'FWD']

    # Perform mutation to generate a mutated league
    mutated_league = deepcopy(league)

    team1, team2 = random.sample(mutated_league.teams, 2)
    position = random.choice(positions)

    players_team1 = [p for p in team1.players if p.position == position]
    players_team2 = [p for p in team2.players if p.position == position]

    if not players_team1 or not players_team2:
        return None  # No valid players to swap

    p1 = random.choice(players_team1)
    p2 = random.choice(players_team2)

    # Perform swap virtually to check
    temp_team1_players = [p for p in team1.players if p != p1] + [p2]
    temp_team2_players = [p for p in team2.players if p != p2] + [p1]

    try:
        # Try creating new Team objects to check for salary cap and full validation
        new_team1 = Team(temp_team1_players)
        new_team2 = Team(temp_team2_players)

        # If both teams are valid, apply changes
        team1.players = new_team1.players
        team2.players = new_team2.players

        # Successful mutation print message
        print(f"Successful mutation swapping {p1.name} ↔ {p2.name} in position {position}")

        return mutated_league  # Valid mutated league created

    except ValueError as e:
        print(f"Invalid mutation")
        return None  # Invalid mutation, return None


In [ ]:
def calculate_fitness(league):
    if league is None or not league.is_valid():
        return 9999  # Penalize invalid league (i.e., failed crossover, mutator)
    
    # Calculate fitness based on standard deviation of average skills
    return league.get_standard_deviation_of_average_skills()


In [ ]:
parent_league = random.choice(population)

child_league = mutate_within_league_by_position(parent_league)

Invalid Crossover


In [336]:
# Print parents' constitution
print("Parent League:")
for i, team in enumerate(parent_league.teams):
    print(f"\nTeam {i+1}:")
    print(team)
    print(f"Average Skill: {team.get_average_skill():.2f}")
print(f"Fitness: {calculate_fitness(parent_league):.2f}")

# Print child league constitution
print("\nChild League:")
if child_league:
    for i, team in enumerate(child_league.teams):
        print(f"\nTeam {i+1}:")
        print(team)
        print(f"Average Skill: {team.get_average_skill():.2f}")
    print(f"Fitness: {calculate_fitness(child_league):.2f}")
else:
    print(f"Fitness: {calculate_fitness(child_league):.2f}")

Parent League:

Team 1:
Alex Carter (GK) Skill: 85 Salary: €90.0M
Mason Reed (DEF) Skill: 82 Salary: €75.0M
Jaxon Griffin (DEF) Skill: 79 Salary: €65.0M
Gavin Richardson (MID) Skill: 87 Salary: €95.0M
Hunter Cooper (MID) Skill: 83 Salary: €85.0M
Chase Murphy (FWD) Skill: 86 Salary: €95.0M
Landon Powell (FWD) Skill: 89 Salary: €110.0M
Average Skill: 84.43

Team 2:
Jordan Smith (GK) Skill: 88 Salary: €100.0M
Owen Parker (DEF) Skill: 88 Salary: €100.0M
Logan Brooks (DEF) Skill: 86 Salary: €95.0M
Dominic Bell (MID) Skill: 86 Salary: €95.0M
Ashton Phillips (MID) Skill: 90 Salary: €110.0M
Julian Scott (FWD) Skill: 92 Salary: €130.0M
Tyler Jenkins (FWD) Skill: 80 Salary: €70.0M
Average Skill: 87.14

Team 3:
Chris Thompson (GK) Skill: 80 Salary: €80.0M
Brayden Hughes (DEF) Skill: 87 Salary: €100.0M
Maxwell Flores (DEF) Skill: 81 Salary: €72.0M
Dylan Morgan (MID) Skill: 91 Salary: €115.0M
Connor Hayes (MID) Skill: 89 Salary: €105.0M
Xavier Bryant (FWD) Skill: 90 Salary: €120.0M
Zachary Nelson (

## NOW CROSSOVER

In [338]:
def crossover_between_teams(league):
    # Select two teams to cross over
    team1, team2 = random.sample(league.teams, 2)
    
    # Perform crossover by swapping player positions, skill levels, etc.
    team1_players = team1.players[:len(team1.players)//2]  # Half from team1
    team2_players = team2.players[len(team2.players)//2:]  # Half from team2
    
    # Combine them into new teams (creating child teams)
    new_team1 = Team(team1_players + team2_players)
    new_team2 = Team(team2_players + team1_players)
    
    # Replace the old teams with the new crossover teams
    league.teams = [new_team1, new_team2]
    
    print(f"Crossover completed between {team1} and {team2}")
    return league  # New league with crossover teams


In [450]:
parent_league = random.choice(population)

# Apply crossover
child_league = crossover_between_teams(deepcopy(parent_league))

# Print Parent League
print("\n=== Parent League ===")
for i, team in enumerate(parent_league.teams):
    print(f"\nTeam {i+1}")
    for player in team.players:
        print(player)
    print(f"Average Skill: {team.get_average_skill():.2f}")
print(f"Fitness: {calculate_fitness(parent_league):.2f}")

# Print Child League
print("\n=== Child League ===")
if child_league:
    for i, team in enumerate(child_league.teams):
        print(f"\nTeam {i+1}")
        for player in team.players:
            print(player)
        print(f"Average Skill: {team.get_average_skill():.2f}")
    print(f"Fitness: {calculate_fitness(child_league):.2f}")
else:
    print("Invalid child league.")
    print(f"Fitness: {calculate_fitness(child_league):.2f}")


Crossover completed between Jordan Smith (GK) Skill: 88 Salary: €100.0M
Maxwell Flores (DEF) Skill: 81 Salary: €72.0M
Brayden Hughes (DEF) Skill: 87 Salary: €100.0M
Nathan Wright (MID) Skill: 92 Salary: €120.0M
Austin Torres (MID) Skill: 82 Salary: €80.0M
Colton Gray (FWD) Skill: 91 Salary: €125.0M
Landon Powell (FWD) Skill: 89 Salary: €110.0M and Chris Thompson (GK) Skill: 80 Salary: €80.0M
Logan Brooks (DEF) Skill: 86 Salary: €95.0M
Daniel Foster (DEF) Skill: 90 Salary: €110.0M
Dominic Bell (MID) Skill: 86 Salary: €95.0M
Gavin Richardson (MID) Skill: 87 Salary: €95.0M
Sebastian Perry (FWD) Skill: 95 Salary: €150.0M
Adrian Collins (FWD) Skill: 85 Salary: €90.0M

=== Parent League ===

Team 1
Ryan Mitchell (GK) Skill: 83 Salary: €85.0M
Ethan Howard (DEF) Skill: 80 Salary: €70.0M
Caleb Fisher (DEF) Skill: 84 Salary: €85.0M
Dylan Morgan (MID) Skill: 91 Salary: €115.0M
Ashton Phillips (MID) Skill: 90 Salary: €110.0M
Elijah Sanders (FWD) Skill: 93 Salary: €140.0M
Zachary Nelson (FWD) Skill